In [3]:
import pandas as pd
import numpy as np
from pyspark import SparkContext
import folium
from folium.plugins import MarkerCluster


In [12]:
#Reading device data from s3 bucket and dispying the same
latlong_data = sqlContext.read.option('delimiter', ' ').option('header','false').csv('s3://ashwini.geolocation/Data_geo/lat_longs.txt')
latlong_list = latlong_data.collect()
#Displaying some records from device file
latlong_list[:10]

[Row(_c0='36.7', _c1='3.216666666666667', _c2='<http://dbpedia.org/resource/Algeria>'),
 Row(_c0='42.5', _c1='1.5166666666666666', _c2='<http://dbpedia.org/resource/Andorra>'),
 Row(_c0='12.516666666666667', _c1='-70.03333333333333', _c2='<http://dbpedia.org/resource/Aruba>'),
 Row(_c0='-8.833333333333334', _c1='13.333333333333334', _c2='<http://dbpedia.org/resource/Angola>'),
 Row(_c0='41.333333333333336', _c1='19.8', _c2='<http://dbpedia.org/resource/Albania>'),
 Row(_c0='34.53333333333333', _c1='69.13333333333334', _c2='<http://dbpedia.org/resource/Afghanistan>'),
 Row(_c0='40.416666666666664', _c1='49.833333333333336', _c2='<http://dbpedia.org/resource/Azerbaijan>'),
 Row(_c0='39.93333333333333', _c1='32.86666666666667', _c2='<http://dbpedia.org/resource/Ankara>'),
 Row(_c0='52.36666666666667', _c1='4.9', _c2='<http://dbpedia.org/resource/Amsterdam>'),
 Row(_c0='50.46', _c1='2.13', _c2='<http://dbpedia.org/resource/Azincourt>')]

In [14]:
#Creating spark dataframe and giving column names
column_name = ['lat', 'long', 'name_of_page']
spark_df = spark.createDataFrame(data = latlong_list, schema = column_name)

In [16]:
#Convert to pandas df
df_panda = spark_df.select("*").toPandas()
df_panda.tail()

lat                long  \
450145  30.002777777777776  -98.40527777777778   
450146             59.7872              5.5104   
450147             59.7823               5.398   
450148             18.7466             84.4219   
450149   39.78111111111111             -87.145   

                                             name_of_page  
450145  <http://dbpedia.org/resource/Twin_Sisters,_Texas>  
450146         <http://dbpedia.org/resource/Stord_Church>  
450147       <http://dbpedia.org/resource/Nysæter_Church>  
450148           <http://dbpedia.org/resource/Pesarapadu>  
450149  <http://dbpedia.org/resource/Adams_Covered_Bri...

In [17]:
#length of df before dropping columns
len(df_panda)

450150

In [19]:
#Dropping rows with zero values
df_panda = df_panda.replace(0, np.nan).dropna(axis=0, how='any').fillna(0)

In [20]:
#length of df before after columns
len(df_panda)

450149

In [21]:
#Creating an empty map and adding to mcluster
world_map= folium.Map(tiles="cartodbpositron")
mcluster = MarkerCluster().add_to(world_map)

In [23]:
latitute_y = df_panda['lat'].tolist()
longitude_x = df_panda['long'].tolist()
for i in range(len(longitude_x)):
    radius=5
    folium.CircleMarker(location = [latitute_y[i], longitude_x[i]], radius=radius, fill =True).add_to(mcluster)

In [ ]:
#Display map data
world_map